## Import required libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Set the random seed

np.random.seed(40)
torch.manual_seed(40)

 ## Data Processing

In [ ]:
# Load the dataset from the specified URL into a pandas DataFrame
data_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df = pd.read_csv(data_url)

In [ ]:
# Extract and reshape the temperature values
temperatures = (
    df.reset_index(drop=True)["Temp"].values.astype(float).reshape(-1, 1))

In [ ]:
# Scale the temperature data to a normalized range [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(temperatures)

In [ ]:
def create_sequences(data, seq_length):
    """ Generates input-output pairs from the provided data."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 20
X, y = create_sequences(data_normalized, seq_length)

## Training and evaluating

In [ ]:
# Split the dataset into training and testing sets with a 67% training size
train_size = int(len(X) * 0.67)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Convert numpy arrays to PyTorch tensors with float32 data type
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## Initiating a LSTM

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.gru(x)
        output = self.fc(output[:, -1, :])
        return output

In [ ]:
# Define hyperparameters for the model training
input_size = 1
hidden_size = 50  # Number of features in the hidden state
output_size = 1
num_layers = 2  # Number of GRU layers
learning_rate = 0.001  # Learning rate
num_epochs = 100
l2_penalty = 0.01  # L2 regularization parameter

In [ ]:
# Instantiate GRU model, and define loss function and optimizer

model = GRUModel(input_size, hidden_size, output_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_penalty)

## Training phase

In [ ]:
# Training loop for the GRU model
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 0.1380
Epoch [2/100], Loss: 0.1100
Epoch [3/100], Loss: 0.0861
Epoch [4/100], Loss: 0.0661
Epoch [5/100], Loss: 0.0498
Epoch [6/100], Loss: 0.0374
Epoch [7/100], Loss: 0.0288
Epoch [8/100], Loss: 0.0240
Epoch [9/100], Loss: 0.0226
Epoch [10/100], Loss: 0.0238
Epoch [11/100], Loss: 0.0263
Epoch [12/100], Loss: 0.0290
Epoch [13/100], Loss: 0.0307
Epoch [14/100], Loss: 0.0312
Epoch [15/100], Loss: 0.0305
Epoch [16/100], Loss: 0.0290
Epoch [17/100], Loss: 0.0271
Epoch [18/100], Loss: 0.0252
Epoch [19/100], Loss: 0.0236
Epoch [20/100], Loss: 0.0225
Epoch [21/100], Loss: 0.0219
Epoch [22/100], Loss: 0.0218
Epoch [23/100], Loss: 0.0219
Epoch [24/100], Loss: 0.0223
Epoch [25/100], Loss: 0.0227
Epoch [26/100], Loss: 0.0231
Epoch [27/100], Loss: 0.0234
Epoch [28/100], Loss: 0.0236
Epoch [29/100], Loss: 0.0237
Epoch [30/100], Loss: 0.0236
Epoch [31/100], Loss: 0.0233
Epoch [32/100], Loss: 0.0230
Epoch [33/100], Loss: 0.0226
Epoch [34/100], Loss: 0.0222
Epoch [35/100], Loss: 0

## Prediction and evaluation

In [ ]:
# Predicting with the model using test data and inverting the scaling of predictions
with torch.no_grad():
    y_pred = model(X_test_tensor)

y_pred_inv = scaler.inverse_transform(y_pred.numpy())

print("Predicted values:", y_pred_inv.squeeze())



Predicted values: [10.708911 10.681672 10.474629 ... 11.387926 11.396119 11.487442]


In [ ]:
# Calculate Mean Squared Error between true and predicted values
mse = mean_squared_error(y_test, y_pred_inv.squeeze())
print("Mean Squared Error:", mse)

Mean Squared Error: 111.89928849851788
